In [1]:
import json
from PrimeNumber import *

In [2]:
print(is_prime(41))

True


# Utility Functions

In [3]:
def complete_with_0(list_input,size):
    res = []
    for elmt in list_input:
        tmp = elmt
        for i in range(len(tmp),size):
            tmp = '0' + tmp
        res.append(tmp)
    return res

def bloc_s1_to_s2(lists1,s1,s2):#Size 1 to size 2
    new_list = []
    index = 0
    tmp = ''
    for elmt in reversed(lists1):
        for i in range(s1):
            tmp = elmt[s1-1-i] + tmp
            index +=1
            if (len(tmp)==s2):
                new_list.insert(0,tmp)
                index=0
                tmp=''
    if (len(tmp) > 0 and len(tmp)<s2):
        cond = False
        for i in range(len(tmp)):
            if tmp[i] != '0':
                cond = True
                break
        if cond :
            for i in range(len(tmp),s2):
                tmp = '0' + tmp
            new_list.insert(0,tmp)
    return new_list

def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  

# Arithmetic Functions

In [4]:
def Extended_Euclidean(a,b):#Suppose a>b
    r, u, v, r2, u2, v2 = a, 1, 0, b, 0,1
    r = a*u + b*v
    r2 = a*u2 + b*v2
    while (r2 != 0):
        q = r//r2
        r, u, v, r2, u2, v2 = r2, u2, v2, r - q *r2, u - q*u2, v - q*v2
        r = a*u + b*v
        r2 = a*u2 + b*v2
    if (v<0):
        v = v + a 
        u = -u - 1
    return(r,u,v)

def find_key(p,q):
    keys = []
    phi = (p-1)*(q-1)
    for i in range(max(p,q)+1,phi):
        if (PGCD(i,phi)==1):
            return i
    return 0
    

Test Extended Euclidean

In [5]:
a,b = 20,3
r,u,v = Extended_Euclidean(a,b)
print('%d = %d.%d + %d.%d' %(r,u,a,v,b))

1 = -1.20 + 7.3


Test find e for RSA

In [6]:
p = 53
q = 97
print(find_key(p,q))

101


# RSA

In [7]:
class RSA():
    
    def __init__(self,p,q):# e primal with n
        cond = True
        if (is_prime_Miller(p,1000) == False):
            print('\033[1m' + 'ERROR \n' + '\033[0m' + '%d is not prime number'%p)
            cond = False
        if (is_prime_Miller(q,1000) == False):
            print('\033[1m' + 'ERROR \n' + '\033[0m' + '%d is not prime number'%q)
            cond = False
        self.p = p
        self.q = q
        self.n = p*q
        self.phi = (p-1)*(q-1)
        self.e = find_key(p,q)
        _,_,self.d = Extended_Euclidean(self.phi,self.e)
        self.bloc = 4
        if cond:
            print("RSA system created with succes")
        
    def public_key(self):
         print('\033[1m' + 'Public Key :' + '\033[0m' +  '\n(n,e) = (%d,%d)'%(self.n,self.e))
        
    def private_key(self):
        print('\033[1m' + 'Private Key :' + '\033[0m' +  '\n(n,d) = (%d,%d)'%(self.n,self.d))
        
    def encrypt(self,M):
        return pow(M,self.e,self.n)
    
    def decrypt(self,C):
        return pow(C,self.d,self.n)
    
    def encrypt_list(self,l):#l a list
        res = []
        for elmt in l:
            res.append(self.encrypt(elmt))
        return res
    
    def decrypt_list(self,l):#l a list
        res = []
        for elmt in l:
            res.append(self.decrypt(elmt))
        return res
    
    def encrypt_text(self,sentence):
        list_tmp = []
        
        #Get the list in ASCII with each element of length 3, ex: '072'
        for char in sentence:
            list_tmp.append(str(ord(char)))
        list_tmp = complete_with_0(list_tmp,3)
        
        #Transform in size bloc, ex bloc=4
        list_tmp = bloc_s1_to_s2(list_tmp,3,self.bloc)
        
        #Transform in int for apply encrypt formulas
        list_res = [int(st) for st in list_tmp] 
        
        #Apply encrypt
        list_res = self.encrypt_list(list_res)
        
        #Return the result in string
        res = [str(i) for i in list_res]
        return res
    
    def decrypt_text(self,sentence):
        
        #Transform in int for apply decrypt formulas
        tmp = [int(i) for i in sentence]
        #Apply decrypt
        tmp = self.decrypt_list(tmp)

        #Get back in string complete with 0
        tmp2 = complete_with_0([str(i) for i in tmp],self.bloc)

        #Get back in size for ASCII
        tmp = bloc_s1_to_s2(tmp2,self.bloc,3)
        
        #Find back the sentence
        res = ""
        for ind in tmp:
            res = res + chr(int(ind))
        return res
    
    def encrypt_file(self,pathfile):
        with open(pathfile,'r') as myfile:
            contents = myfile.read()
        res = self.encrypt_text(contents)
        with open('message_encrypt.txt','w') as myfile:
            json.dump(res,myfile)
        return None
    
    def decrypt_file(self,pathfile):
        with open(pathfile,'r') as myfile:
            contents = json.load(myfile)
        res = self.decrypt_text(contents)
        with open('message_decrypt.txt','w') as myfile:
            myfile.write(res)
        

In [16]:
p = pow(2,607) - 1
q = pow(2,521) - 1
#p = 379
#q = 467
print('Size of p: %d' %len(str(p)))
print('Size of q: %d' %len(str(q)))
rsa = RSA(p,q)
rsa.public_key()
rsa.private_key()

Size of p: 183
Size of q: 157
RSA system created with succes
Public Key :
(n,e) = (3646154850295011369707131011438711095400799139943170490872585628683549034362552065955809589514611470241298944167703929337528884908857116141935206466329731087514964112054543019336536216107629523597606330154669196064144182472739556974502462402438903115845725630946428943768540714098264727068026730424033578827886916761701429264950573899186177,531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728131)
Private Key :
(n,d) = (3646154850295011369707131011438711095400799139943170490872585628683549034362552065955809589514611470241298944167703929337528884908857116141935206466329731087514964112054543019336536216107629523597606330154669196064144182472739556974502462402438903115845725630946428943768540714098264727068026730424033578827886916761701429264950573899186177,158922059681823943838269434

In [10]:
sentence = 'Help !'
print(sentence)

Help !


In [11]:
res = rsa.encrypt_text(sentence)
print("res : ",res)

res :  ['57663', '19946', '96710', '11273', '128600']


In [12]:
res_inv = rsa.decrypt_text(res)
print(res_inv)

Help !


In [13]:
print('Succes: ',sentence == res_inv)

Succes:  True


In [17]:
rsa.encrypt_file('message_clean.txt')

In [18]:
rsa.decrypt_file('message_encrypt.txt')